In [ ]:
import psi4
import numpy as np
from helper_CC_ML_old import *
import os
import pandas as pd
MLt2=0

In [ ]:
#This box contains all the features the CCSD module logs for us to make predictions
#Ultimately, just make sure you run this every time and all is well.

features = ['Evir1', 'Hvir1', 'Jvir1', 'Kvir1', 'Evir2', 'Hvir2', 'Jvir2', 'Kvir2', 'Eocc1', 'Jocc1', 'Kocc1', 'Hocc1',
            'Eocc2', 'Jocc2', 'Kocc2', 'Hocc2', 'Jia1', 'Jia2', 'Kia1', 'Kia2', 'diag', 'orbdiff', 'doublecheck', 't2start', 't2mag', 't2sign', 'Jia1mag', 'Jia2mag', 'Kia1mag', 'Kia2mag']
'''
Key:
Letters:
E-Energy of the orbital
H-1e contribution to the orbital energy
J-Coulombic contribution to orbital energy
K-Exchange contribution to orbital energy
Placement:
occ or virt, you get this..
Number:
is it electron one or two from the two electron excitation


Jia1- coulomb integral between orbital occ1 and vir1
Jia2 " but 2
Kia1 - exchange integral between orbital 
Kia2 Same but exchange integral
diag - is it on the diagonal, aka, are the two excited electrons going to the same orbital **this is important fyi
orbdiff - (Evir2 + Evir1 - Eocc1 - Eocc2)
doublecheck - full 2electron integral
t2start - INITIAL MP2 amplitude **this is the inital guess
t2mag - np.log10(np.absolute(t2start)) ~ this is going to be a common trend, since it is more straightforward for ML algorithms to understand
t2sign - (t2start > 1)? 
Jia1mag - np.log10(np.absolute(feature))
Jia2mag np.log10(np.absolute(feature))
Kia1mag  np.log10(np.absolute(feature))
Kia2mag np.log10(np.absolute(feature))

'''

#numbah=18
#print (steps[numbah,:])
#print (features[numbah])
factors=(1,1.25,1.5,2,5,10,100,1000)
#Evir1
factor= np.zeros((len(features)))
finalfactor=factor
factor[2]=3
factor[6]=3
factor[12]=4
factor[8]=4
factor[16]=1
factor[17]=1
factor[20]=5
factor[21]=5
factor[22]=6
factor[23]=6
factor[24]=5
factor[25]=1
factor=factor.astype(int)
#print (feature[8])
for j in range(0,len(features)):
    a=factor[j]
    finalfactor[j]=factors[a]


In [ ]:
#This function will extract all the features and CCSD t2 amplitudes from a file with xyz or z-matrix of a molecule

def GetFeatures(File_path, occ=False, vir=False):
    i=1
    Features_file = open('Molecule_features.csv', 'w+')
    Features_file.write('Evir1, Hvir1, Jvir1, Kvir1, Evir2, Hvir2, Jvir2, Kvir2, Eocc1, Jocc1, Kocc1, Hocc1, Eocc2, Jocc2, Kocc2, Hocc2, Jia1, Jia2, Kia1, Kia2, diag, orbdiff, doublecheck, t2start, t2mag, t2sign, Jia1mag, Jia2mag, Kia1mag, Kia2mag, CCSD-t2\n')
    text = open(File_path, 'r').read()
    mol = psi4.geometry(text)
    
    psi4.core.clean()
    psi4.set_options({'basis':        'cc-pVDZ',
                      'scf_type':     'pk',
                      'reference':    'rhf',
                      'mp2_type':     'conv',
                      'e_convergence': 1e-8,
                      'd_convergence': 1e-8})
        
    A=HelperCCEnergy(mol)

    A.compute_energy()
    matrixsize=A.nocc*A.nocc*A.nvirt*A.nvirt
    Bigmatrix=np.zeros([matrixsize, len(features)])
    for x in range(0,len(features)):
        Bigmatrix[:, x]=getattr(A, features[x]).reshape(matrixsize)
    Bigamp=A.t2.reshape(matrixsize,1)
    if i==1:
        Bigfeatures=Bigmatrix
        Bigamps=Bigamp
        i=2
    else:
        Bigfeatures=np.vstack((Bigfeatures,Bigmatrix))
        Bigamps=np.vstack((Bigamps,Bigamp))
    
# Load data to the .csv file    
    for amp in range(0, Bigfeatures.shape[0]):
        for feature in range(0, Bigfeatures.shape[1]):
            Features_file.write(F'{Bigfeatures[amp][feature]},')
        Features_file.write(F'{Bigamps[amp][0]}\n')
    
    Features_file.close()

In [ ]:
# Extracting features and amplitudes from Water_example.dat file
#If the molecule file is in another location use the path to the file

GetFeatures('Water/Regular/Water5/Water1.xyz')